<a href="https://colab.research.google.com/github/KartikayBhardwaj-dev/Ai_lab/blob/main/ai_lab_week05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hugging face


#INTRO
- hugging face is a platform where large datasets, pretrained models, are there
- in this we import large wikipedia text and then tokenise them
- tokenise (tokenizer) helps to convert text into numerical inputs

#IMPORT DATASETS

In [ ]:
from datasets import load_dataset
#load a raw text dataset

raw_data = load_dataset("wikitext", "wikitext-2-raw-v1")


README.md: 0.00B [00:00, ?B/s]

wikitext-2-raw-v1/test-00000-of-00001.pa(…):   0%|          | 0.00/733k [00:00<?, ?B/s]

wikitext-2-raw-v1/train-00000-of-00001.p(…):   0%|          | 0.00/6.36M [00:00<?, ?B/s]

wikitext-2-raw-v1/validation-00000-of-00(…):   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

#FINE TUNING GPT2

Gpt2 is a decoder only transformer to predict next word in sentence
- change to gpu then install libraries

In [3]:
!pip install --upgrade transformers

In [4]:
!pip install datasets accelerate evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.4 MB/s eta 0:00:00


#LOAD DATATSET FROM DATATSETS ( HUGGING FACE )

In [2]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    Trainer,
    AutoModelForCausalLM
)

- datasets is the hugging face library to load datasets like WikiText easily
- tansformer gives us:
1. tokenizer: turns text into token Ids that model understand
2. model: GPT-2
3. DataCollator: handles batch formatting and padding
4. trainingArguments and trainer: simplify traiing loops
5. why important: without these, you'd have to write yourn data loader, optimizer, evaluation loop

#1. Load And Prepare dataset

In [5]:
#load only 5000 examples from the training split, loading from hugging face
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train[:5000]")

#remove empty lines (common in wikitext data)
dataset = dataset.filter(lambda x: len(x["text"]) > 0)

#create train/validation split 90% train, 10% validation
split = dataset.train_test_split(test_size=0.1, seed=42)
train_raw = split["train"]
val_raw = split["test"]


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

wikitext-2-raw-v1/test-00000-of-00001.pa(…):   0%|          | 0.00/733k [00:00<?, ?B/s]

wikitext-2-raw-v1/train-00000-of-00001.p(…):   0%|          | 0.00/6.36M [00:00<?, ?B/s]

wikitext-2-raw-v1/validation-00000-of-00(…):   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5000 [00:00<?, ? examples/s]

#2. Load tokenizer and model

In [6]:
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

#gpt 2 has no pad token we add one for batching
if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({"pad_token": "[PAD]"})  #pad is technique to give id to data labels

model = AutoModelForCausalLM.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(50258, 768)

tokenizer maps words -> integers using gpt2 vocab

gpt2 doesnt have a padding token by default but batching need it. so we add one

model is gpt2 with a language modelling head

resize token embeddings ensures the model knows our new pad token

if skipped:
wihout padding batches of different lenghts will crash

without resizing embeddings you ll get a size mismatch error

e

#3.Tokenize the text, convertng text into tensors

In [9]:
max_length = 512
def tokenize_text(examples):
  return tokenizer(examples["text"], truncation=True, max_length=max_length)

train_tok = train_raw.map(tokenize_text, batched=True, remove_columns=["text"])
val_tok = val_raw.map(tokenize_text, batched=True, remove_columns=["text"])
#make datasets convert to pytorch tensors
train_tok.set_format(type="torch", columns=["input_ids", "attention_mask"])
val_tok.set_format(type="torch", columns=["input_ids", "attention_mask"])

Map:   0%|          | 0/2904 [00:00<?, ? examples/s]

Map:   0%|          | 0/323 [00:00<?, ? examples/s]

- truncation = true: cuts long text at max_length tokens
- map applies our tokenizer to whole dataset
- remove_columns: drops the raw text after tokenizatio to save memory

#4.Create the datacollator

In [12]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

the collator batches tokenized data and pads sequences in the batches to same length mlm=False means casual LM(predict next token) not masked LM like bert

if wrong

setting mlm=True would train gpt2 as bert style training

#5.Finetuning

5. Define training arguments

In [14]:
training_args = TrainingArguments(
    output_dir="./gpt2-wikitext-finetuned", #save model checkpoints
    num_train_epochs=10,
    per_device_train_batch_size=4,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_steps=500,
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    load_best_model_at_end=True,
    save_total_limit=3,
    fp16=True,
    report_to="none"
)

For model performance
- learning_rate: a small 5e-5 is common to start with
- weight_decay: a regularisation technique to prevent the model from becoming too complex and overfitiing tot the training data

for logging , saving , evaluation
- evaluation_strategy: when to perform evaluation set to "steps" or "epochs"
- eval_steps: if using evaluation_strategy="steps", this sets how often to run evaluation
- same for save
- load_best_model: trainer will load best performing model at end

for speed and efficiency
- fp16: set to true enables mixed precision training
reduce gpu memory

6. Trainer

In [15]:
import transformers
print(transformers.__version__)
from transformers import TrainingArguments
print(TrainingArguments.__module__)



4.57.1
transformers.training_args


In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    data_collator=data_collator,
    processing_class=tokenizer,
)

In [21]:
#train

import time
#start time
start_time = time.perf_counter()
#train

trainer.train()
#end timer
end_time = time.perf_counter()

#calculate
elapsed_time = end_time - start_time
minutes, seconds = divmod(elapsed_time, 60)
print(f"Total training time: {int(minutes)} min {seconds:.2f} sec")

Step,Training Loss,Validation Loss


KeyboardInterrupt: 

#Finetuning bert model for sentiment analysis

What is sentiment analysis
- technique used to determine tone of sequence
- fine tuning a encoder based transformer DistilBert

Import

In [2]:
!pip install transformers datasets evaluate accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.8 MB/s eta 0:00:00


In [3]:
import numpy as np
import evaluate
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)

AutoTokenizer: choose correct tokenizer for chosen model
AutoModelForSequenceClassification: loads pre trained model with a sequence classification head on top this head is a simple linear layer that we will fine tune for our sentiment task

Load and prepare data

In [4]:
# we will use imdb dataset, which contains 50000 movies reviews labelled as either positive (1) or negative (0)
#load dataset
raw_dataset = load_dataset("imdb")

#create smaller set
small_train_dataset = raw_dataset["train"].shuffle(seed=42).select(range(1000))
small_test_dataset = raw_dataset["test"].shuffle(seed=42).select(range(1000))
print(f"training data sample \n {small_train_dataset[0]}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

training data sample 
 {'text': 'There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier\'s plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it\'s the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...', 'label': 1}


Load tokenizer and model

In [5]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tokenize the text

In [6]:
def tokenize_function(examples):
  return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_train_dataset = small_train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = small_test_dataset.map(tokenize_function, batched=True)



Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

padding=max_length : pads all sentences to models maximum sequence length this ensure all batches have the same size

truncation:true trancates any longer review than models maximum length

Define Metrics and training arguments

In [7]:
#define metric
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

#define training arguments
training_args = TrainingArguments(
    output_dir="sentiment_model",
    eval_strategy="epoch",
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=5e-5,
    weight_decay=0.01,
    fp16=True,
    report_to="none"
)

Create and run trainer


In [8]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    compute_metrics=compute_metrics,
)
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.391184,0.852000
2,No log,0.399307,0.868000


TrainOutput(global_step=250, training_loss=0.34120352172851565, metrics={'train_runtime': 45.0427, 'train_samples_per_second': 44.402, 'train_steps_per_second': 5.55, 'total_flos': 264934797312000.0, 'train_loss': 0.34120352172851565, 'epoch': 2.0})

testing

In [14]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis", model=trainer.model, tokenizer=tokenizer, device=0)

print(sentiment_pipeline("failed in exams"))
print(sentiment_pipeline("the plot was unpredictable"))

Device set to use cuda:0


[{'label': 'LABEL_0', 'score': 0.8847382068634033}]
[{'label': 'LABEL_0', 'score': 0.9644890427589417}]
